In [1]:
# %matplotlib ipympl

In [4]:
# %matplotlib inline

In [1]:
%matplotlib tk

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import colors
from matplotlib.widgets import Button, Slider

In [5]:
# Displacements from a cell to its eight nearest neighbours
neighbourhood = ((-1,-1), (-1,0), (-1,1), (0,-1), (0, 1), (1,-1), (1,0), (1,1))
EMPTY, TREE, FIRE = 0, 1, 2
# Colours for visualization: brown for EMPTY, dark green for TREE and orange
# for FIRE. Note that for the colormap to work, this list and the bounds list
# must be one larger than the number of different values in the array.
colors_list = [(0.2,0,0), (0,0.5,0), (1,0,0), 'orange']
cmap = colors.ListedColormap(colors_list)
bounds = [0,1,2,3]
norm = colors.BoundaryNorm(bounds, cmap.N)

In [7]:
# The initial fraction of the forest occupied by trees.
forest_fraction = 0.65
# Probability of new tree growth per empty cell, and of lightning strike.
# p, f = 0.05, 0.0001
# here adjusted to align with lecture example
p = 0
f = 0
# Forest size (number of cells in x and y directions).
nx, ny = 100, 100

In [9]:
def iterate(X):
    """Iterate the forest according to the forest-fire rules."""

    # The boundary of the forest is always empty, so only consider cells
    # indexed from 1 to nx-2, 1 to ny-2
    X1 = np.zeros((ny, nx))
    for ix in range(1,nx-1):
        for iy in range(1,ny-1):
            if X[iy,ix] == EMPTY and np.random.random() <= p:
                X1[iy,ix] = TREE
            if X[iy,ix] == TREE:
                X1[iy,ix] = TREE
                for dx,dy in neighbourhood:
                    # The diagonally-adjacent trees are further away, so
                    # only catch fire with a reduced probability:
                    if abs(dx) == abs(dy) and np.random.random() < 0.573:
                        continue
                    if X[iy+dy,ix+dx] == FIRE:
                        X1[iy,ix] = FIRE
                        break
                else:
                    if np.random.random() <= f:
                        X1[iy,ix] = FIRE
    return X1


In [11]:
# Initialize the forest grid.
X  = np.zeros((ny, nx))

def init(forest_fraction):
    global X
    X[1:ny-1, 1:nx-1] = np.random.randint(0, 2, size=(ny-2, nx-2))
    X[1:ny-1, 1:nx-1] = np.random.random(size=(ny-2, nx-2)) < forest_fraction

In [13]:
def animate(i):
    im.set_data(animate.X)
    animate.X = iterate(animate.X)

In [15]:
def simulation(density):
    global fig, ax, bnext, bstart, bstop, binit, time, anim,im
    # apparently the buttons need to be declared as global to avoid 
    # that they are garbage-collected. If this happens they will still be visible but become inactive

    def advance(d):
        global time, im
        time += 1
        im.set_data(animate.X)
        animate.X = iterate(animate.X)
        plt.title('t = ' + str(time))    
        fig.canvas.draw_idle()

    def stopAnim(d):
        global anim
        anim.pause()

    def startAnim(d):
        global anim
        anim.resume()

    def initAnim(d):
        global X, TREE, FIRE,im
        global mat, plt, time
        global density
        time = 0
        init(density)
        animate.X = X
        started = False
        while not started:
            xs = np.random.randint(0,nx-1)
            ys = np.random.randint(0,ny-1)
            if X[ys,xs] == TREE:
                X[ys,xs] = FIRE
                started = True
        im.set_data(X)
        plt.title('t = ' + str(time))    
        fig.canvas.draw_idle() 

    def updateDensity(val):
        global density
        density = sdensity.val

    plt.close('all')
    fig, ax = plt.subplots()
    ax.axis('off')
    plt.title("Forest Fire Percolation Model")

    axdensity = plt.axes([0.175, 0.05, 0.65, 0.03])
    sdensity = Slider(axdensity, 'Threshold', 0.0, 1.0, valinit=density)
    sdensity.on_changed(updateDensity)

    axnext = plt.axes([0.85, 0.15, 0.1, 0.075])
    axstart = plt.axes([0.85, 0.25, 0.1, 0.075])
    axstop = plt.axes([0.85, 0.35, 0.1, 0.075])
    axinit = plt.axes([0.85, 0.45, 0.1, 0.075])
    bnext = Button(axnext, 'Next')
    bnext.on_clicked(advance)
    bstart = Button(axstart, 'Resume')
    bstart.on_clicked(startAnim)
    bstop = Button(axstop, 'Stop')
    bstop.on_clicked(stopAnim)
    binit = Button(axinit, 'Init')
    binit.on_clicked(initAnim)
    im = ax.imshow(X, cmap=cmap, norm=norm)

    initAnim(None)
    anim = animation.FuncAnimation(fig, advance, frames=None, cache_frame_data=False, interval=60, repeat=False)

In [17]:
density=0.7
threshold =0.7
simulation(density)